In [ ]:
# 150TX ResNet18 循环SNR 全日期闭集（block-level 样本独立） 解决了训练集和测试集的样本不均衡
from joblib import load
import pandas as pd
import numpy as np
import os
from data_utilities import *
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
from tqdm.auto import tqdm
from collections import defaultdict

import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt
import math

# ------------------------ 数据集加载 ------------------------
dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])

# ------------------------ 去掉不参与训练的 TX ------------------------
tx_list = compact_dataset['tx_list']
exclude_txs = ['12-1', '18-1', '19-4', '20-3', '7-12', '9-14']
tx_list = [tx for tx in tx_list if tx not in exclude_txs]

print(f"✅ 剩余用于训练的 TX 数量: {len(tx_list)}")
print(f"具体为：{tx_list}")

rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']
print("闭集合训练，所有日期为：", capture_date_list)

n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)

# ------------------------ 参数设置 ------------------------
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 180        # 每个 block 的信号数
y = 10                  # 拼接时每组多少条信号
test_ratio = 0.25       # 测试集比例

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
FS = 20e6
FC = 2.4e9
SNR_DB = -10
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True
BATCH_SIZE = 64
EPOCHS = 200
LR = 1e-3
WEIGHT_DECAY = 5e-3
N_SPLITS = 5
PATIENCE = 10
MIN_DELTA = 0.05
in_planes    = 64
dropout      = 0.7
SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)
script_name = "wisig_XFR"

# ------------------------ 调用数据处理函数 ------------------------
train_blocks, train_block_labels, test_blocks, test_block_labels = preprocess_dataset_cross_IQ_balanced_blocks_per_tx_day(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("train_blocks shape:", train_blocks.shape)
print("test_blocks  shape:", test_blocks.shape) 

# ---------- 可选：AWGN / Doppler ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()
    
# ---------- Block 互斥检查函数 ----------
def check_block_overlap(tr_blocks, va_blocks, test_blocks, fold_num=None):
    """
    检查训练集、验证集、测试集 block 是否存在重复
    tr_blocks / va_blocks / test_blocks: np.array [N_block, block_size, L, 2]
    """
    def blocks_to_set(blocks):
        return set([b.tobytes() for b in blocks])

    tr_set = blocks_to_set(tr_blocks)
    va_set = blocks_to_set(va_blocks)
    test_set = blocks_to_set(test_blocks)

    overlap_tr_va = tr_set & va_set
    overlap_tr_test = tr_set & test_set
    overlap_va_test = va_set & test_set

    if overlap_tr_va or overlap_tr_test or overlap_va_test:
        msg = f"❌ Block overlap detected"
        if fold_num is not None:
            msg += f" in fold {fold_num}"
        print(msg)
        if overlap_tr_va: print(f"Train/Val overlap: {len(overlap_tr_va)} blocks")
        if overlap_tr_test: print(f"Train/Test overlap: {len(overlap_tr_test)} blocks")
        if overlap_va_test: print(f"Val/Test overlap: {len(overlap_va_test)} blocks")
        raise ValueError("Block overlap detected! Aborting training.")
    else:
        print(f"✅ Fold {fold_num}: Train/Val/Test blocks are mutually exclusive.")
        
# ------------------------ 数据标准化函数 ------------------------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    修复版：
    - 噪声只在能量归一化+Doppler 之后加入
    - 删除 per-sample 零均值/单位方差归一化（会洗掉噪声）
    - 完全保持你原本的数据流格式（最小化改动）
    """

    # ---- 展开 block ----
    if data_real_imag.ndim == 4:  
        N_block, block_size, L, C = data_real_imag.shape
        data_real_imag = data_real_imag.reshape(N_block*block_size, L, C)
    else:
        N_block, L, C = data_real_imag.shape

    data = data_real_imag.astype(np.float32).copy()
    out = np.empty_like(data, dtype=np.float32)

    for i in range(len(data)):
        iq = data[i]
        sigc = iq[...,0] + 1j * iq[...,1]

        # -------- 1) 能量归一化（保留你的配置） --------
        sigc = sigc / (np.sqrt(np.mean(np.abs(sigc)**2)) + 1e-12)

        # -------- 2) Doppler shift --------
        if add_doppler:
            sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)

        # -------- 3) AWGN —— 正确位置（不会被洗掉） --------
        if add_noise:
            sigc = add_complex_awgn(sigc, snr_db)

        # -------- 4) 不再做 per-sample 标准化 --------
        iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)
        out[i] = iq

    return torch.tensor(out, dtype=torch.float32)

# === ResNet 1D 模型定义 ===
class BasicBlock1D(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout=0.0):
        super().__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)

class ResNet18_1D_Dynamic(nn.Module):
    def __init__(self, num_classes=10, in_planes=64, dropout=0.5, input_length=256):
        super().__init__()
        self.in_planes = in_planes
        self.conv1 = nn.Conv1d(2, in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1, dropout=dropout)
        self.layer2 = self._make_layer(128, 2, stride=2, dropout=dropout)
        self.layer3 = self._make_layer(256, 2, stride=2, dropout=dropout)
        self.layer4 = self._make_layer(512, 2, stride=2, dropout=dropout)

        # 动态计算 flatten 后长度
        L = input_length
        for stride in [2,2,2,2,2]:  # conv1 stride2 + maxpool stride2 + layer2/3/4 stride2
            L = (L + stride - 1) // stride  # ceil division
        self.flattened_length = 512 * L

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_length, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def _make_layer(self, planes, blocks, stride, dropout):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes)
            )
        layers = [BasicBlock1D(self.in_planes, planes, stride, downsample, dropout)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock1D(self.in_planes, planes, dropout=dropout))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [B,L,2] -> [B,2,L]
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return self.fc(x)


# ------------------------ K-Fold 训练（block-level展开样本） ------------------------
def train_kfold_pointcloud(train_blocks, train_block_labels, test_blocks, test_block_labels,
                           num_classes, device=DEVICE, batch_size=BATCH_SIZE, epochs=EPOCHS,
                           lr=LR, weight_decay=WEIGHT_DECAY, n_splits=N_SPLITS,
                           patience=PATIENCE, min_delta=MIN_DELTA, script_name=script_name):
    """
    train_blocks: [N_block, block_size, 256, 2]
    train_block_labels: [N_block]
    """
    input_length = train_blocks.shape[2]  # 动态获取 block 内信号长度 L
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}dB_fd{fd}_classes_{num_classes}_ResNet"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # ------------------------ 测试集 loader ------------------------
    X_test_torch = preprocess_for_pointcloud_cnn(
        test_blocks,
        add_noise=ADD_NOISE,
        snr_db=SNR_DB,
        add_doppler=ADD_DOPPLER,
        fd_hz=fd
    )
    y_test_torch = torch.tensor(test_block_labels, dtype=torch.long).repeat_interleave(test_blocks.shape[1])
    test_dataset = TensorDataset(X_test_torch, y_test_torch)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=2, pin_memory=True)


    # ------------------------ K-Fold ------------------------
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(train_blocks))
    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")

        # 训练/验证集 block
        tr_blocks = train_blocks[tr_idx]; tr_labels = train_block_labels[tr_idx]
        va_blocks = train_blocks[va_idx]; va_labels = train_block_labels[va_idx]

        # ---------- 检查 block 互斥 ----------
        check_block_overlap(tr_blocks, va_blocks, test_blocks, fold_num=fold+1)

        # 展开 block -> 样本
        X_tr = preprocess_for_pointcloud_cnn(
            tr_blocks,
            add_noise=ADD_NOISE,
            snr_db=SNR_DB,
            add_doppler=ADD_DOPPLER,
            fd_hz=fd
        )
        y_tr = torch.tensor(tr_labels, dtype=torch.long).repeat_interleave(tr_blocks.shape[1])
        X_va = preprocess_for_pointcloud_cnn(
            va_blocks,
            add_noise=ADD_NOISE,
            snr_db=SNR_DB,
            add_doppler=ADD_DOPPLER,
            fd_hz=fd
        )
        y_va = torch.tensor(va_labels, dtype=torch.long).repeat_interleave(va_blocks.shape[1])

        tr_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(TensorDataset(X_va, y_va), batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        # ------------------------ 模型初始化 ------------------------
        model = ResNet18_1D_Dynamic(
            num_classes=num_classes,
            in_planes=in_planes,
            dropout=dropout,
            input_length=input_length
            ).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0; best_wts = None; patience_cnt = 0
        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            # ---------- 训练 ----------
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms: total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p==yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad,1); avg_grad_list.append(avg_grad)
            train_losses.append(running_loss/len(tr_loader)); train_accs.append(100.*correct/total)

            # ---------- 验证 ----------
            model.eval()
            vloss, vcorrect, vtotal = 0.0,0,0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out,1)
                    vcorrect += (p==yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy())
                    all_preds.extend(p.cpu().numpy())
            val_losses.append(vloss/len(va_loader)); val_accs.append(100.*vcorrect/max(1,vtotal))
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_accs[-1]:.2f}% | ValAcc={val_accs[-1]:.2f}% | "
                  f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file,'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_accs[-1]:.2f}% | ValAcc={val_accs[-1]:.2f}% | "
                        f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad:.4f}\n")

            if val_accs[-1] > best_val + min_delta:
                best_val = val_accs[-1]; best_wts = model.state_dict(); patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping."); break
            scheduler.step()

        # ---------- restore best ----------
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # ---------- 训练/验证/测试评估 ----------
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)

        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)

        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')

        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file,'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file,'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    return save_folder

# ------------------------ 批量 SNR 实验 ------------------------
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE
    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    SNR_DB = snr_db
    ADD_NOISE = True

    num_classes = len(np.unique(train_block_labels))
    save_folder = train_kfold_pointcloud(train_blocks, train_block_labels, test_blocks, test_block_labels,
                                         num_classes=num_classes, script_name=script_name)
    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder

snr_list = list(range(-10, -41, -5))
all_results = {}
for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")


In [ ]:
# 150TX ResNet18 循环SNR 全日期闭集（block-level 样本独立）
from joblib import load
import pandas as pd
import numpy as np
import os
from data_utilities import *
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
from tqdm.auto import tqdm
from collections import defaultdict

import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt
import math

# ------------------------ 数据集加载 ------------------------
dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']
print("闭集合训练，所有日期为：", capture_date_list)

n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# ------------------------ 参数设置 ------------------------
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                   # 拼接时每组多少条信号
test_ratio = 0.25       # 测试集比例

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
FS = 20e6
FC = 2.4e9
SNR_DB = -10
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True
BATCH_SIZE = 256
EPOCHS = 200
LR = 1e-4
WEIGHT_DECAY = 1e-3
N_SPLITS = 5
PATIENCE = 5
MIN_DELTA = 0.1
SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)
script_name = "wisig_XFR"

# ------------------------ 调用数据处理函数 ------------------------
train_blocks, train_block_labels, test_blocks, test_block_labels = preprocess_dataset_cross_IQ_independent_blocks_per_tx_day(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("train_blocks shape:", train_blocks.shape)
print("test_blocks  shape:", test_blocks.shape) 

# ---------- 可选：AWGN / Doppler ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()
    
# ---------- Block 互斥检查函数 ----------
def check_block_overlap(tr_blocks, va_blocks, test_blocks, fold_num=None):
    """
    检查训练集、验证集、测试集 block 是否存在重复
    tr_blocks / va_blocks / test_blocks: np.array [N_block, block_size, L, 2]
    """
    def blocks_to_set(blocks):
        return set([b.tobytes() for b in blocks])

    tr_set = blocks_to_set(tr_blocks)
    va_set = blocks_to_set(va_blocks)
    test_set = blocks_to_set(test_blocks)

    overlap_tr_va = tr_set & va_set
    overlap_tr_test = tr_set & test_set
    overlap_va_test = va_set & test_set

    if overlap_tr_va or overlap_tr_test or overlap_va_test:
        msg = f"❌ Block overlap detected"
        if fold_num is not None:
            msg += f" in fold {fold_num}"
        print(msg)
        if overlap_tr_va: print(f"Train/Val overlap: {len(overlap_tr_va)} blocks")
        if overlap_tr_test: print(f"Train/Test overlap: {len(overlap_tr_test)} blocks")
        if overlap_va_test: print(f"Val/Test overlap: {len(overlap_va_test)} blocks")
        raise ValueError("Block overlap detected! Aborting training.")
    else:
        print(f"✅ Fold {fold_num}: Train/Val/Test blocks are mutually exclusive.")
        
# ------------------------ 数据标准化函数 ------------------------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: 
        [N, L, 2] 或 [N_block, block_size, L, 2]
    输出:
        torch.tensor [N_total, L, 2]
    关键修复:
        - 去掉零均值/单位方差 normalize，避免把噪声洗掉
        - 噪声放到 Doppler 后，并保留其能量差异
    """

    # ---- 展开 block ----
    if data_real_imag.ndim == 4:  
        N_block, block_size, L, C = data_real_imag.shape
        data_real_imag = data_real_imag.reshape(N_block*block_size, L, C)
    else:
        N_block, L, C = data_real_imag.shape

    data = data_real_imag.astype(np.float32).copy()
    out = np.empty_like(data, dtype=np.float32)

    for i in range(len(data)):
        iq = data[i]
        sigc = iq[...,0] + 1j * iq[...,1]

        # -------- 1) 能量归一化 (保留你的原逻辑) --------
        sigc = sigc / (np.sqrt(np.mean(np.abs(sigc)**2)) + 1e-12)

        # -------- 2) Doppler shift --------
        if add_doppler:
            sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)

        # -------- 3) AWGN (修复: 噪声作用被保留) --------
        if add_noise:
            sigc = add_complex_awgn(sigc, snr_db)

        # -------- 4) 不再做 per-sample 标准化 --------
        iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        out[i] = iq

    return torch.tensor(out, dtype=torch.float32)


# ------------------------ RF1DCNN ------------------------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
    def forward(self, x):
        identity = x
        out = self.conv1(x); out = self.bn1(out); out = self.relu(out)
        out = self.conv2(out); out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(identity)
        out += identity
        out = self.relu(out)
        return out

class RF1DCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3, input_length=256):
        super().__init__()
        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7); self.pool1 = nn.MaxPool1d(2)
        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5); self.pool2 = nn.MaxPool1d(2)
        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5); self.pool3 = nn.MaxPool1d(2)
        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3); self.pool4 = nn.MaxPool1d(2)

        # 动态计算 flatten 后长度
        L = input_length
        for _ in range(4):  # 4个 MaxPool1d(2)
            L = L // 2
        self.flattened_length = 256 * L

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_length, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = x.permute(0,2,1)  # [B,L,2] -> [B,2,L]
        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)
        return self.fc(x)

# ------------------------ K-Fold 训练（block-level展开样本） ------------------------
def train_kfold_pointcloud(train_blocks, train_block_labels, test_blocks, test_block_labels,
                           num_classes, device=DEVICE, batch_size=BATCH_SIZE, epochs=EPOCHS,
                           lr=LR, weight_decay=WEIGHT_DECAY, n_splits=N_SPLITS,
                           patience=PATIENCE, min_delta=MIN_DELTA, script_name=script_name):
    """
    train_blocks: [N_block, block_size, 256, 2]
    train_block_labels: [N_block]
    """
    input_length = train_blocks.shape[2]  # 动态获取 block 内信号长度 L
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}dB_fd{fd}_classes_{num_classes}_ResNet"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # ------------------------ 测试集 loader ------------------------
    X_test_torch = preprocess_for_pointcloud_cnn(
        test_blocks,
        add_noise=ADD_NOISE,
        snr_db=SNR_DB,
        add_doppler=ADD_DOPPLER,
        fd_hz=fd
    )
    y_test_torch = torch.tensor(test_block_labels, dtype=torch.long).repeat_interleave(test_blocks.shape[1])
    test_dataset = TensorDataset(X_test_torch, y_test_torch)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # ------------------------ K-Fold ------------------------
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(train_blocks))
    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")

        # 训练/验证集 block
        tr_blocks = train_blocks[tr_idx]; tr_labels = train_block_labels[tr_idx]
        va_blocks = train_blocks[va_idx]; va_labels = train_block_labels[va_idx]

        # ---------- 检查 block 互斥 ----------
        check_block_overlap(tr_blocks, va_blocks, test_blocks, fold_num=fold+1)

        # 展开 block -> 样本
        X_tr = preprocess_for_pointcloud_cnn(
            tr_blocks,
            add_noise=ADD_NOISE,
            snr_db=SNR_DB,
            add_doppler=ADD_DOPPLER,
            fd_hz=fd
        )
        y_tr = torch.tensor(tr_labels, dtype=torch.long).repeat_interleave(tr_blocks.shape[1])
        X_va = preprocess_for_pointcloud_cnn(
            va_blocks,
            add_noise=ADD_NOISE,
            snr_db=SNR_DB,
            add_doppler=ADD_DOPPLER,
            fd_hz=fd
        )
        y_va = torch.tensor(va_labels, dtype=torch.long).repeat_interleave(va_blocks.shape[1])

        tr_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(TensorDataset(X_va, y_va), batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        # ------------------------ 模型初始化 ------------------------
        model = RF1DCNN(num_classes=num_classes, dropout=0.3, input_length=input_length).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0; best_wts = None; patience_cnt = 0
        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            # ---------- 训练 ----------
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms: total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p==yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad,1); avg_grad_list.append(avg_grad)
            train_losses.append(running_loss/len(tr_loader)); train_accs.append(100.*correct/total)

            # ---------- 验证 ----------
            model.eval()
            vloss, vcorrect, vtotal = 0.0,0,0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out,1)
                    vcorrect += (p==yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy())
                    all_preds.extend(p.cpu().numpy())
            val_losses.append(vloss/len(va_loader)); val_accs.append(100.*vcorrect/max(1,vtotal))
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_accs[-1]:.2f}% | ValAcc={val_accs[-1]:.2f}% | "
                  f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file,'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_accs[-1]:.2f}% | ValAcc={val_accs[-1]:.2f}% | "
                        f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad:.4f}\n")

            if val_accs[-1] > best_val + min_delta:
                best_val = val_accs[-1]; best_wts = model.state_dict(); patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping."); break
            scheduler.step()

        # ---------- restore best ----------
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # ---------- 训练/验证/测试评估 ----------
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)

        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)

        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')

        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file,'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file,'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    return save_folder

# ------------------------ 批量 SNR 实验 ------------------------
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE
    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    SNR_DB = snr_db
    ADD_NOISE = True

    X_train_torch = preprocess_for_pointcloud_cnn(train_blocks, add_noise=True, snr_db=snr_db, add_doppler=ADD_DOPPLER)
    X_test_torch = preprocess_for_pointcloud_cnn(test_blocks, add_noise=True, snr_db=snr_db, add_doppler=ADD_DOPPLER)

    num_classes = len(np.unique(train_block_labels))
    save_folder = train_kfold_pointcloud(train_blocks, train_block_labels, test_blocks, test_block_labels,
                                         num_classes=num_classes, script_name=script_name)
    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder

snr_list = list(range(20, -41, -5))
all_results = {}
for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")


In [ ]:
# 150TX ResNet18 循环SNR 全日期闭集（block-level 样本独立） 解决了训练集和测试集的样本不均衡
from joblib import load
import pandas as pd
import numpy as np
import os
from data_utilities import *
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
from tqdm.auto import tqdm
from collections import defaultdict

import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt
import math

# ------------------------ 数据集加载 ------------------------
dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])

# ------------------------ 去掉不参与训练的 TX ------------------------
tx_list = compact_dataset['tx_list']
exclude_txs = ['12-1', '18-1', '19-4', '20-3', '7-12', '9-14']
tx_list = [tx for tx in tx_list if tx not in exclude_txs]

print(f"✅ 剩余用于训练的 TX 数量: {len(tx_list)}")
print(f"具体为：{tx_list}")

rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']
print("闭集合训练，所有日期为：", capture_date_list)

n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)

# ------------------------ 参数设置 ------------------------
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 180        # 每个 block 的信号数
y = 10                  # 拼接时每组多少条信号
test_ratio = 0.25       # 测试集比例
EPOCHS = 200

# ------------------------ 调用数据处理函数 ------------------------
train_blocks, train_block_labels, test_blocks, test_block_labels = preprocess_dataset_cross_IQ_balanced_blocks_per_tx_day(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from itertools import product, islice
from datetime import datetime
from sklearn.metrics import confusion_matrix
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------ 设置设备 ------------------------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ------------------------ 随机选择 X 个组合 ------------------------
param_grid = {
    'batch_size': [64, 128, 256],
    'learning_rate': [1e-3, 1e-4, 5e-4],
    'weight_decay': [1e-3, 5e-3],
    'in_planes': [32, 64],
    'dropout': [0.3, 0.5, 0.7]
}
NUM_COMBINATIONS =  10  # 抽取 X 种组合

# 生成所有组合
all_combinations = list(product(
    param_grid['batch_size'],
    param_grid['learning_rate'],
    param_grid['weight_decay'],
    param_grid['in_planes'],
    param_grid['dropout']
))
np.random.seed(42)
selected_combinations = np.random.choice(len(all_combinations), NUM_COMBINATIONS, replace=False)
selected_combinations = [all_combinations[i] for i in selected_combinations]

# ------------------------ 模型定义 ------------------------
class BasicBlock1D(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout=0.0):
        super().__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)

class ResNet18_1D_Dynamic(nn.Module):
    def __init__(self, num_classes=10, in_planes=64, dropout=0.5, input_length=256):
        super().__init__()
        self.in_planes = in_planes
        self.conv1 = nn.Conv1d(2, in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2, stride=1, dropout=dropout)
        self.layer2 = self._make_layer(128, 2, stride=2, dropout=dropout)
        self.layer3 = self._make_layer(256, 2, stride=2, dropout=dropout)
        self.layer4 = self._make_layer(512, 2, stride=2, dropout=dropout)

        # 动态计算 flatten 长度
        L = input_length
        for stride in [2,2,2,2,2]:  # conv1+maxpool+layer2/3/4
            L = (L + stride - 1)//stride
        self.flattened_length = 512 * L

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_length, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def _make_layer(self, planes, blocks, stride, dropout):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes)
            )
        layers = [BasicBlock1D(self.in_planes, planes, stride, downsample, dropout)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock1D(self.in_planes, planes, dropout=dropout))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.permute(0,2,1)  # [B,L,2] -> [B,2,L]
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return self.fc(x)

# ------------------------ 数据预处理函数 ------------------------
def preprocess_for_pointcloud_cnn(data_real_imag):
    if data_real_imag.ndim == 4:  
        N_block, block_size, L, C = data_real_imag.shape
        data_real_imag = data_real_imag.reshape(N_block*block_size, L, C)
    data = data_real_imag.astype(np.float32).copy()
    iq = data[...,0] + 1j*data[...,1]
    iq = iq / (np.sqrt(np.mean(np.abs(iq)**2))+1e-12)
    data = np.stack([np.real(iq), np.imag(iq)], axis=-1).astype(np.float32)
    return torch.tensor(data, dtype=torch.float32)

# ------------------------ 评估函数 ------------------------
def evaluate_model(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out,1)
            correct += (p==yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(len(set(all_labels)))))
    return acc, cm

# ------------------------ 训练函数（带早停） ------------------------
def train_one_combination(train_blocks, train_labels, test_blocks, test_labels,
                          batch_size, lr, weight_decay, in_planes, dropout, epochs=50,
                          patience=5):
    
    input_length = train_blocks.shape[2]
    num_classes = len(np.unique(train_labels))
    
    X_train = preprocess_for_pointcloud_cnn(train_blocks)
    y_train = torch.tensor(train_labels, dtype=torch.long).repeat_interleave(train_blocks.shape[1])
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

    X_test = preprocess_for_pointcloud_cnn(test_blocks)
    y_test = torch.tensor(test_labels, dtype=torch.long).repeat_interleave(test_blocks.shape[1])
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)
    
    model = ResNet18_1D_Dynamic(num_classes=num_classes, in_planes=in_planes, dropout=dropout, input_length=input_length).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    best_acc = 0.0
    patience_counter = 0
    
    for epoch in range(epochs):
        # -------- 训练 --------
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_loss = running_loss / len(train_loader)
        
        # -------- 测试集评估 --------
        train_acc, _ = evaluate_model(model, train_loader, DEVICE)
        test_acc, _ = evaluate_model(model, test_loader, DEVICE)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss={train_loss:.4f} | Train Acc={train_acc:.2f}% | Test Acc={test_acc:.2f}%")
        
        # -------- 早停逻辑 --------
        if test_acc > best_acc:
            best_acc = test_acc
            best_wts = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1} as test accuracy did not improve for {patience} epochs.")
                break
    
    model.load_state_dict(best_wts)
    return model, best_acc

# ------------------------ 执行随机组合训练（保持原网格搜索逻辑） ------------------------
SAVE_ROOT = "./search_results"
os.makedirs(SAVE_ROOT, exist_ok=True)
grid_results = []

for idx, (batch_size, lr, weight_decay, in_planes, dropout) in enumerate(selected_combinations, 1):
    print(f"\n{'='*60}\n🔹 Training combination {idx}/{NUM_COMBINATIONS}\nBatch={batch_size}, LR={lr}, WD={weight_decay}, In_planes={in_planes}, Dropout={dropout}\n{'='*60}")
    model, best_acc = train_one_combination(train_blocks, train_block_labels, test_blocks, test_block_labels,
                                            batch_size, lr, weight_decay, in_planes, dropout,
                                            epochs=EPOCHS, patience=5)
    # 保存模型
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_file = os.path.join(SAVE_ROOT, f"model_comb{idx}_{timestamp}.pth")
    torch.save(model.state_dict(), model_file)
    grid_results.append({
        'combination_idx': idx,
        'batch_size': batch_size,
        'lr': lr,
        'weight_decay': weight_decay,
        'in_planes': in_planes,
        'dropout': dropout,
        'best_test_acc': best_acc,
        'model_file': model_file
    })

# ------------------------ 保存总结 ------------------------
summary_file = os.path.join(SAVE_ROOT, f'grid_search_summary_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')
with open(summary_file, 'w') as f:
    json.dump(grid_results, f, indent=4)
print(f"\n✅ Training finished. Summary saved to {summary_file}")


In [ ]:
# 150TX ResNet18 循环SNR 全日期闭集 block会被拆分
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])
tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']
print("闭集合训练，所有日期为：", capture_date_list)

n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


# 参数设置
max_sig = None          # 每个 TX-RX-日期最多使用的信号数
block_size = 240        # 每个 block 的信号数
y = 5                   # 拼接时每组多少条信号
test_ratio = 0.25       # 测试集比例

# 调用函数
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_all_mix_random(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    max_sig=max_sig,
    equalized=equalized,
    block_size=block_size,
    y=y,
    test_ratio=test_ratio,
    seed=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# ---------- PyTorch & 训练配置 ----------
import torch
torch._dynamo.disable()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime
import pywt  # 保留，若需扩展
import math

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 数据增强 / 信道设置（目前默认关闭）
FS = 20e6
FC = 2.4e9
SNR_DB = -10
VELOCITY_KMH = 120
ADD_NOISE = True
ADD_DOPPLER = True

# 训练参数
BATCH_SIZE = 256   # 更大 batch 更稳定（你数据很多）
EPOCHS = 1
LR = 1e-3
WEIGHT_DECAY = 1e-4
N_SPLITS = 5
PATIENCE = 8
MIN_DELTA = 0.1   # early stopping 的最小进步（%）

SAVE_ROOT = "./training_results"
os.makedirs(SAVE_ROOT, exist_ok=True)

script_name = "wisig_XFR"

# ---------- 可选：AWGN / Doppler ----------
def compute_doppler_shift(v_kmh, fc_hz):
    if not v_kmh:
        return 0.0
    c = 3e8
    v = v_kmh / 3.6
    return fc_hz * v / c

def add_complex_awgn(signal, snr_db):
    if snr_db is None:
        return signal
    power = np.mean(np.abs(signal)**2)
    noise_power = power / (10**(snr_db/10))
    noise_std = np.sqrt(noise_power/2)
    noise = noise_std * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))
    return signal + noise

def apply_doppler_shift(signal, fd_hz, fs_hz):
    if fd_hz is None or fd_hz == 0:
        return signal
    t = np.arange(len(signal)) / fs_hz
    return signal * np.exp(1j * 2 * np.pi * fd_hz * t)

# ---------- 数据预处理：针对交错样本，用 per-sample 标准化 ----------
def preprocess_for_pointcloud_cnn(data_real_imag, add_noise=False, snr_db=None,
                                  add_doppler=False, fd_hz=None, fs_hz=FS):
    """
    输入 data_real_imag: np.array [N, L, 2] (I,Q)
    输出 torch.tensor [N, L, 2] (float32)，每个样本做 zero-mean unit-std 标准化（按样本）
    """
    data = data_real_imag.astype(np.float32).copy()
    N, L, C = data.shape
    out = np.empty_like(data, dtype=np.float32)
    for i in range(N):
        iq = data[i]  # (L,2)
        # 组合成 complex
        sigc = iq[...,0] + 1j * iq[...,1]

        # step1 归一化功率
        sigc = sigc / (np.sqrt(np.mean(np.abs(sigc)**2)) + 1e-12)

        # step2 可选多普勒
        if add_doppler:
            sigc = apply_doppler_shift(sigc, fd_hz, fs_hz)

        # step3 可选噪声
        if add_noise:
            sigc = add_complex_awgn(sigc, snr_db)

        # 转回 IQ
        iq = np.stack([np.real(sigc), np.imag(sigc)], axis=-1).astype(np.float32)

        # per-sample 标准化
        mu = iq.mean(axis=(0,))
        sigma = iq.std(axis=(0,))
        sigma[sigma < 1e-8] = 1.0
        iq_norm = (iq - mu) / sigma
        out[i] = iq_norm

    return torch.tensor(out, dtype=torch.float32)

# ---------- 新的 RF1DCNN ----------
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.downsample = None
        if in_channels != out_channels or stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        return out


class RF1DCNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super().__init__()

        self.layer1 = ResidualBlock1D(2, 32, kernel_size=7)
        self.pool1 = nn.MaxPool1d(2)  # 240 → 120

        self.layer2 = ResidualBlock1D(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(2)  # 120 → 60

        self.layer3 = ResidualBlock1D(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool1d(2)  # 60 → 30

        self.layer4 = ResidualBlock1D(128, 256, kernel_size=3)
        self.pool4 = nn.MaxPool1d(2)  # 30 → 15

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 15, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # x: (B, 240, 2)
        x = x.permute(0, 2, 1)  # → (B, 2, 240)

        x = self.layer1(x); x = self.pool1(x)
        x = self.layer2(x); x = self.pool2(x)
        x = self.layer3(x); x = self.pool3(x)
        x = self.layer4(x); x = self.pool4(x)

        return self.fc(x)


# ---------- 辅助绘图/评估函数 ----------
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    correct, total = 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            _, p = torch.max(out, 1)
            correct += (p == yb).sum().item()
            total += yb.size(0)
            all_labels.extend(yb.cpu().numpy())
            all_preds.extend(p.cpu().numpy())
    acc = 100.0 * correct / total if total>0 else 0.0
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
    return acc, cm

def plot_confusion_matrix(cm, classes, fold, save_folder, dataset_type='Test'):
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{dataset_type} Confusion Matrix Fold{fold}')
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.savefig(os.path.join(save_folder, f'{dataset_type.lower()}_cm_fold{fold}.png'))
    plt.close()

def plot_curves(train_losses, val_losses, train_acc, val_acc, fold, save_folder):
    plt.figure(); plt.plot(train_losses, label='Train Loss'); plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Fold {fold} Loss'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'loss_fold{fold}.png')); plt.close()
    plt.figure(); plt.plot(train_acc, label='Train Acc'); plt.plot(val_acc, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.title(f'Fold {fold} Accuracy'); plt.legend(); plt.grid(True)
    plt.savefig(os.path.join(save_folder, f'acc_fold{fold}.png')); plt.close()

# ---------- K-Fold 训练主函数（使用 IQPointCloudCNN） ----------
def train_kfold_pointcloud(X_train, y_train, X_test, y_test, num_classes, device=DEVICE,
                           batch_size=BATCH_SIZE, epochs=EPOCHS, lr=LR, weight_decay=WEIGHT_DECAY,
                           n_splits=N_SPLITS, patience=PATIENCE, min_delta=MIN_DELTA,
                           script_name=script_name):
    fd = int(compute_doppler_shift(VELOCITY_KMH, FC))
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    save_dir = f"{timestamp}_{script_name}_SNR{SNR_DB}dB_fd{fd}_classes_{num_classes}_ResNet"
    save_folder = os.path.join(SAVE_ROOT, save_dir)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    # 保存参数
    with open(results_file, 'w') as f:
        f.write("=== Experiment Parameters ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"SNR_dB: {SNR_DB}, ADD_NOISE: {ADD_NOISE}, ADD_DOPPLER: {ADD_DOPPLER}\n")
        f.write(f"FS: {FS}, FC: {FC}, Velocity_kmh: {VELOCITY_KMH}\n")
        f.write(f"Batch: {batch_size}, Epochs: {epochs}, LR: {lr}, WD: {weight_decay}\n")
        f.write(f"Num classes: {num_classes}, K-Fold: {n_splits}, Patience: {patience}, MinDelta: {min_delta}\n")
        f.write("============================\n\n")

    # test loader
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # full dataset for K-Fold
    full_dataset = TensorDataset(X_train, y_train)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    indices = np.arange(len(full_dataset))

    val_scores, test_scores = [], []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(indices)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        tr_sub = Subset(full_dataset, tr_idx)
        va_sub = Subset(full_dataset, va_idx)
        tr_loader = DataLoader(tr_sub, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        va_loader = DataLoader(va_sub, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

        model = RF1DCNN(num_classes=num_classes, dropout=0.3).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        best_val = 0.0
        best_wts = None
        patience_cnt = 0

        train_losses, val_losses, train_accs, val_accs = [], [], [], []
        avg_grad_list = []

        for epoch in range(epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            total_grad, cnt_grad = 0.0, 0
            for xb, yb in tr_loader:
                xb = xb.to(device); yb = yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                # grad norm
                grad_norms = [p.grad.norm().item() for p in model.parameters() if p.grad is not None]
                if grad_norms:
                    total_grad += np.mean(grad_norms); cnt_grad += 1
                optimizer.step()
                running_loss += loss.item()
                _, p = torch.max(out, 1)
                correct += (p == yb).sum().item()
                total += yb.size(0)
            avg_grad = total_grad / max(cnt_grad, 1)
            avg_grad_list.append(avg_grad)

            train_loss = running_loss / max(1, len(tr_loader))
            train_acc = 100.0 * correct / max(1, total)
            train_losses.append(train_loss); train_accs.append(train_acc)

            # validation
            model.eval()
            vloss, vcorrect, vtotal = 0.0, 0, 0
            all_labels, all_preds = [], []
            with torch.no_grad():
                for xb, yb in va_loader:
                    xb = xb.to(device); yb = yb.to(device)
                    out = model(xb)
                    loss = criterion(out, yb)
                    vloss += loss.item()
                    _, p = torch.max(out, 1)
                    vcorrect += (p == yb).sum().item()
                    vtotal += yb.size(0)
                    all_labels.extend(yb.cpu().numpy()); all_preds.extend(p.cpu().numpy())
            val_loss = vloss / max(1, len(va_loader))
            val_acc = 100.0 * vcorrect / max(1, vtotal)
            val_losses.append(val_loss); val_accs.append(val_acc)
            val_cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
            np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)

            print(f"Epoch {epoch+1}/{epochs} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                  f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}")
            with open(results_file, 'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_acc:.2f}% | ValAcc={val_acc:.2f}% | "
                        f"TrainLoss={train_loss:.4f} | ValLoss={val_loss:.4f} | AvgGrad={avg_grad:.4f}\n")

            # Early stopping on validation accuracy with min_delta (percentage points)
            if val_acc > best_val + min_delta:
                best_val = val_acc
                best_wts = model.state_dict()
                patience_cnt = 0
            else:
                patience_cnt += 1
                if patience_cnt >= patience:
                    print("Early stopping.")
                    break

            scheduler.step()

        # restore best
        if best_wts is not None:
            model.load_state_dict(best_wts)

        # 保存 train/val 混淆矩阵
        train_acc, train_cm = evaluate_model(model, tr_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'train_cm_fold{fold+1}.npy'), train_cm)
        plot_confusion_matrix(train_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Train')

        val_acc, val_cm = evaluate_model(model, va_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'val_cm_fold{fold+1}.npy'), val_cm)
        plot_confusion_matrix(val_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Val')

        # test eval
        test_acc, test_cm = evaluate_model(model, test_loader, device, num_classes)
        np.save(os.path.join(save_folder, f'test_cm_fold{fold+1}.npy'), test_cm)
        plot_confusion_matrix(test_cm, classes=list(range(num_classes)), fold=fold+1, save_folder=save_folder, dataset_type='Test')
        print(f"Fold {fold+1} Test Accuracy: {test_acc:.2f}%")
        with open(results_file, 'a') as f:
            f.write(f"Fold{fold+1} TestAcc={test_acc:.2f}%\n")

        # 保存曲线 & 模型
        plot_curves(train_losses, val_losses, train_accs, val_accs, fold+1, save_folder)
        torch.save(model.state_dict(), os.path.join(save_folder, f'model_fold{fold+1}.pth'))

        val_scores.append(val_acc)
        test_scores.append(test_acc)

    # summary
    print("\n=== Overall Summary ===")
    print(f"Val Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}")
    print(f"Test Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}")
    with open(results_file, 'a') as f:
        f.write(f"\n=== Overall Summary ===\nVal Acc: {np.mean(val_scores):.2f} ± {np.std(val_scores):.2f}\nTest Acc: {np.mean(test_scores):.2f} ± {np.std(test_scores):.2f}\n")

    print(f"\nAll results saved in {save_folder}")
    return save_folder

# ---------- 运行前的准备（预处理并转换为张量） ----------
# X_train, X_test 目前 shape: [N, 240, 2] (numpy)
# 我们对每个样本做 per-sample 标准化（zero-mean unit-std）
print("Preprocessing (per-sample normalization)...")
# ---------- 转为 torch.tensor 并做 per-sample 标准化 ----------

X_train_torch = preprocess_for_pointcloud_cnn(X_train, add_noise=False)
X_test_torch  = preprocess_for_pointcloud_cnn(X_test, add_noise=False)

y_train_torch = torch.tensor(y_train, dtype=torch.long)
y_test_torch  = torch.tensor(y_test, dtype=torch.long)

num_classes = len(torch.unique(y_train_torch))
print("Prepared tensors:", X_train_torch.shape, X_test_torch.shape, "num_classes=", num_classes)

# ---------- K-Fold 训练 ----------
# train_kfold_pointcloud 内部会把训练集再划分为训练+验证集
# save_folder = train_kfold_pointcloud(
#    X_train_torch, 
#    y_train_torch,
#    X_test_torch, 
#    y_test_torch,
#    num_classes=num_classes
# )

#print("Done. Results in:", save_folder)

##########################################################
#=== 定义一个统一入口函数，用于跑一次 SNR 实验 ===
##########################################################
def run_experiment_with_snr(snr_db):
    global SNR_DB, ADD_NOISE

    print("\n" + "="*60)
    print(f"🚀 Running experiment with SNR = {snr_db} dB")
    print("="*60)

    # 设置当前实验 SNR
    SNR_DB = snr_db
    ADD_NOISE = True   # 打开 AWGN
    
    # --- 重新预处理数据（会调用 add_noise=True） ---
    X_train_torch_snr = preprocess_for_pointcloud_cnn(
        X_train, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )
    X_test_torch_snr = preprocess_for_pointcloud_cnn(
        X_test, add_noise=True, add_doppler = ADD_DOPPLER, snr_db=snr_db
    )

    y_train_torch_snr = torch.tensor(y_train, dtype=torch.long)
    y_test_torch_snr  = torch.tensor(y_test,  dtype=torch.long)

    num_classes = len(torch.unique(y_train_torch_snr))



    # --- 调用你的 K-Fold 训练 ---
    save_folder = train_kfold_pointcloud(
        X_train_torch_snr,
        y_train_torch_snr,
        X_test_torch_snr,
        y_test_torch_snr,
        num_classes=num_classes,
        script_name=script_name 

    )

    print(f"🎉 Finished SNR={snr_db} dB, results saved in: {save_folder}")
    return save_folder


###########################################################
# === 批量执行 SNR Sweep: 20, 10, 0, -10, ... -40 dB ===
###########################################################
snr_list = list(range(20, -41, -5))
# [20, 15, 10, 5, 0, -5, -10, -15, -20, -25, -30, -35, -40]

all_results = {}

for snr in snr_list:
    folder = run_experiment_with_snr(snr)
    all_results[snr] = folder

print("\n\n================ FINAL SUMMARY ================")
for snr, folder in all_results.items():
    print(f"SNR {snr:>3} dB → results in: {folder}")
print("==============================================")

